In [34]:
import pandas as pd
import numpy as np
import math
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from collections import defaultdict
from sklearn.preprocessing import LabelEncoder
import nltk

In [35]:
data = pd.read_csv('../Data/preprocessed.csv')

In [36]:
reviews = data['review'].values
labels = data['sentiment'].values
encoder = LabelEncoder()
encoded_labels = encoder.fit_transform(labels)

In [37]:
encoded_labels

array([1, 1, 1, ..., 0, 0, 0], dtype=int64)

In [38]:
train_text, test_text, train_label, test_label = train_test_split(data['review'], encoded_labels, stratify = encoded_labels)

In [41]:
vec = CountVectorizer(max_features = 3000)
x = vec.fit_transform(train_text)
vocab = vec.get_feature_names()
x = x.toarray()

c:\Users\rashmi\AppData\Local\pypoetry\Cache\virtualenvs\text-toxicity-detection-BTqULhGY-py3.9\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [42]:
# word_counts = {}
# word_counts[0] = {word: sum(x[:][word]) for word in vocab}

word_counts = {}
for l in range(2):
    word_counts[l] = defaultdict(lambda: 0)
for i in range(x.shape[0]):
    l = train_label[i]
    for j in range(len(vocab)):
        word_counts[l][vocab[j]] += x[i][j]

In [39]:
# v = list(map(sum,zip(*x)))
# d = dict(zip(vocab, v))

# word_counts = {0:d, 1:d}

In [45]:
def laplace_smoothing(n_label_items, vocab, word_counts, word, text_label):
    a = word_counts[text_label][word] + 1
    b = n_label_items[text_label] + len(vocab)
    return math.log(a/b)

In [46]:
def group_by_label(x, y, labels):
    data = {}
    for l in labels:
        data[l] = x[np.where(y == l)]
    return data

In [47]:
def fit(x, y, labels):
    n_label_items = {}
    log_label_priors = {}
    n = len(x)
    grouped_data = group_by_label(x, y, labels)
    for l, data in grouped_data.items():
        n_label_items[l] = len(data)
        log_label_priors[l] = math.log(n_label_items[l] / n)
    return n_label_items, log_label_priors

In [48]:
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()

In [49]:
def predict(n_label_items, vocab, word_counts, log_label_priors, labels, x):
    result = []
    for text in x:
        label_scores = {l: log_label_priors[l] for l in labels}
        words = set(w_tokenizer.tokenize(text))
        for word in words:
            if word not in vocab: continue
            for l in labels:
                log_w_given_l = laplace_smoothing(n_label_items, vocab, word_counts, word, l)
                label_scores[l] += log_w_given_l
        result.append(max(label_scores, key=label_scores.get))
    return result

In [50]:
labels = [0,1]
n_label_items, log_label_priors = fit(train_text,train_label,labels)
pred = predict(n_label_items, vocab, word_counts, log_label_priors, labels, test_text)
print("Accuracy of prediction on test set : ", accuracy_score(test_label,pred))

KeyError: 'key of type tuple not found and not a MultiIndex'

In [44]:
train_text[np.where(train_label == 0)]

KeyError: 'key of type tuple not found and not a MultiIndex'

In [63]:
train_text[list(np.where(train_label == 0)[0])]

KeyError: '[3, 7, 16, 18, 21, 37, 52, 54, 62, 66, 76, 114, 126, 132, 139, 141, 147, 151, 155, 156, 160, 169, 190, 200, 204, 214, 223, 224, 227, 232, 234, 247, 257, 284, 299, 304, 306, 308, 321, 325, 327, 328, 332, 334, 347, 349, 350, 357, 374, 380, 382, 384, 391, 395, 398, 404, 406, 410, 432, 438, 440, 464, 469, 473, 474, 476, 477, 491, 494, 515, 539, 545, 549, 554, 555, 558, 562, 566, 570, 577, 615, 618, 626, 636, 637, 642, 651, 652, 664, 684, 689, 691, 695, 705, 715, 724, 728, 732, 733, 758, 780, 783, 795, 805, 808, 809, 828, 833, 852, 857, 868, 870, 880, 888, 892, 907, 925, 956, 960, 963, 980, 988, 998, 1006, 1008, 1009, 1020, 1027, 1038, 1054, 1062, 1065, 1069, 1087, 1089, 1112, 1132, 1154, 1157, 1165, 1166, 1173, 1181, 1183, 1195, 1203, 1204, 1209, 1216, 1222, 1224, 1228, 1229, 1244, 1250, 1253, 1254, 1255, 1256, 1272, 1277, 1284, 1294, 1301, 1302, 1306, 1308, 1320, 1326, 1331, 1341, 1377, 1386, 1396, 1402, 1405, 1428, 1433, 1435, 1437, 1438, 1446, 1448, 1457, 1469, 1470, 1480, 14